In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trl
#!pip install -q -U git+https://github.com/huggingface/accelerate.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install -U bitsandbytes

In [5]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,              # 리포지토리의 custom 코드를 실행하도록 설정
    quantization_config=quant_config,      # BitsAndBytesConfig 객체 전달
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [7]:
model

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSdpaAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (c_fc_1): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (c_proj): Linear4bit(in_features=14336, out_features=4096, bias=

In [7]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Exaone 모델 구조에 맞게 target_modules 목록을 수정
lora_config = LoraConfig(
    r=4,                     # LoRA 가중치 행렬의 rank (값이 작을수록 trainable parameter 수가 줄어듦)
    lora_alpha=8,            # LoRA 스케일링 팩터
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'out_proj', 'c_fc_0', 'c_fc_1', 'c_proj'],
    bias='none',             # bias 파라미터 학습 여부: 'none', 'all', 'lora_only'
    task_type="CAUSAL_LM"     # 문자열로 task_type 지정 (이전에는 TaskType.CAUSAL_LM을 사용했음)
)

# 양자화된 모델 학습 전 전처리
model = prepare_model_for_kbit_training(model)

# PEFT를 사용해 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()


trainable params: 10,485,760 || all params: 7,828,934,656 || trainable%: 0.1339


In [8]:
train = pd.read_csv("/content/drive/MyDrive/train.csv", encoding = 'utf-8-sig')

In [9]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

In [38]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [39]:
from datasets import Dataset

# combined_training_data가 Pandas DataFrame인 경우
dataset_hf = Dataset.from_pandas(combined_training_data.reset_index(drop=True))


In [40]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

In [41]:
def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["question"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"text": prompt, "answer": data_point["answer"]}

# 기존 데이터셋의 모든 컬럼을 제거하고, generate_prompt에서 필요한 컬럼만 남깁니다.
remove_column_keys = list(dataset_hf.features.keys())
dataset_cvted = dataset_hf.shuffle(seed=42).map(generate_prompt, remove_columns=remove_column_keys)

Map:   0%|          | 0/23422 [00:00<?, ? examples/s]

In [42]:
dataset_cvted

Dataset({
    features: ['answer', 'text'],
    num_rows: 23422
})

In [43]:
print(dataset_cvted[0])

{'answer': '작업 중 안전한 작업방법에 대한 지속적인 교육 실시와 작업 전 작업방법 교육을 포함한 철저한 현장 안전관리.', 'text': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n작업 프로세스는 '설치작업'이며 기타발생, 사고 원인은 '작업자가 올바른 작업방법 모름, 안전수칙 미준수'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}


In [44]:
# 토크나이징 완료된 데이터셋을 train/test로 분리
split_datasets = dataset_cvted.train_test_split(test_size=0.1, seed=42)

train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]  # 보통 검증 용도로 사용

In [45]:
train_dataset[0]

{'answer': '작업자 안전교육 및 각종 공구사용법 숙지교육 실시 계획.',
 'text': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n작업 프로세스는 '연결작업'이며 기타발생, 사고 원인은 '철근삽입용 구멍천공중 함마드릴을 손에서 놓침'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}

In [46]:
def tokenize_function(examples):
    outputs = tokenizer(examples["text"], truncation=True, max_length=512)
    return outputs


In [47]:
remove_column_keys = train_dataset.features.keys()
train_dataset_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

remove_column_keys = eval_dataset.features.keys()
test_dataset_tokenized = eval_dataset.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

Map:   0%|          | 0/21079 [00:00<?, ? examples/s]

Map:   0%|          | 0/2343 [00:00<?, ? examples/s]

In [48]:
train_dataset_tokenized

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 21079
})

In [49]:
test_dataset_tokenized

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2343
})

In [50]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [51]:
# 2. TrainingArguments에 evaluation_strategy와 eval_steps 추가
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=8,         # 각 디바이스당 배치 사이즈
    gradient_accumulation_steps=4,
    warmup_steps=1,
    max_steps=2000,
    learning_rate=2e-4,                     # 학습률
    bf16=True,                              # bf16 사용 (하드웨어 지원 확인 필요)
    output_dir="/content/drive/MyDrive/model_weight",
    optim="paged_adamw_8bit",               # 8비트 AdamW 옵티마이저
    logging_steps=50,                       # 로깅 빈도
    save_total_limit=3,                     # 저장할 체크포인트의 최대 수
    evaluation_strategy="steps",            # 일정 스텝마다 평가 실행
    eval_steps=50                           # 평가 주기 (50 스텝마다 평가)
)

# 3. SFTTrainer에 eval_dataset을 추가하여 수정
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=train_args,
    data_collator=collate_fn
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/21079 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/21079 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/21079 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/21079 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/2343 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/2343 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2343 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2343 [00:00<?, ? examples/s]

In [52]:
model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mhwooya7777 (mhwooya7777-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,1.068500,0.439611
100,0.416700,0.419350
150,0.406200,0.410514
200,0.406200,0.404891
250,0.393200,0.399807
300,0.400200,0.396798
350,0.403100,0.394600
400,0.383800,0.393291
450,0.382900,0.390794
500,0.378900,0.389087


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


KeyboardInterrupt: 

In [53]:
# 저장 - > 이때 qLoRA를 이용해서 학습한 파라미터만 저장되서 작음
FINETUNED_MODEL = "qlora"
trainer.model.save_pretrained(FINETUNED_MODEL)

In [54]:
from peft import PeftConfig

In [55]:
#불러오기
FINETUNED_MODEL = "/content/qlora"

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [56]:
# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path
)

The repository for LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [57]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [58]:
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# 샘플 TEST
prompt = dataset_cvted[2]['text']
# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=64)
outputs = pipe(
    prompt,
    do_sample=False,
    temperature=0.1,
    #top_k=0,
    #top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


안전 교육 강화와 함께 작업 현장의 미끄러짐 위험 지역에 경고 표지판 설치하고 정기적인 점검과 개인 보호 장비 착용 의무화로 개인 주의력 향상 유도합니다._향후 지속 가능성 검토 위한 월간 안전성 평가 실시 예정 입니다_.


In [59]:
dataset_cvted[2]

{'answer': '작업절차 및 말비계 사용에 관한 안전교육 실시와 작업 전 안전교육 실시 요청.',
 'text': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n작업 프로세스는 '설치작업'이며 넘어짐(미끄러짐)발생, 사고 원인은 '개인 부주의'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}

In [ ]:
# test 데이터 전처리
import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

# 1. CSV 파일 로드 (예: test.csv)
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")

# 데이터 전처리
test_df['공사종류(대분류)'] = test_df['공사종류'].str.split(' / ').str[0]
test_df['공사종류(중분류)'] = test_df['공사종류'].str.split(' / ').str[1]
test_df['공종(대분류)'] = test_df['공종'].str.split(' > ').str[0]
test_df['공종(중분류)'] = test_df['공종'].str.split(' > ').str[1]
test_df['사고객체(대분류)'] = test_df['사고객체'].str.split(' > ').str[0]
test_df['사고객체(중분류)'] = test_df['사고객체'].str.split(' > ').str[1]

# 2. train 데이터와 동일한 방식으로 prompt 생성
def generate_prompt_from_row(row):
    prompt = (
         f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
    )
    return {"prompt": prompt}

# DataFrame의 각 행에 대해 prompt 생성 (새로운 열로 추가)
test_df = test_df.apply(generate_prompt_from_row, axis=1)
test_df = pd.DataFrame(test_df.tolist())

# 3. HuggingFace Dataset으로 변환
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["prompt"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"prompt": prompt,}

remove_column_keys = list(test_dataset.features.keys())
test_dataset = test_dataset.map(generate_prompt, remove_columns=remove_column_keys)

Map:   0%|          | 0/964 [00:00<?, ? examples/s]

In [95]:
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

# 가정: test_dataset은 HuggingFace Dataset 객체이며, merged_model과 tokenizer가 이미 정의되어 있음.
# 예시: test_dataset.features = ['prompt']

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=64)

# 배치 사이즈 설정 (메모리 상황에 맞게 조정)
batch_size = 8
test_results = []

# HuggingFace Dataset은 slicing 시 dict 형태로 각 컬럼을 반환할 수 있음
for i in tqdm(range(0, len(test_dataset), batch_size)):
    # i부터 i+batch_size 까지 슬라이싱하여 배치 데이터 추출
    batch = test_dataset[i:i+batch_size]
    # "prompt" 컬럼의 리스트를 추출 (예: batch["prompt"]가 리스트 형태)
    batch_prompts = batch["prompt"]

    # 배치로 텍스트 생성 (한 번에 여러 프롬프트 처리)
    batch_outputs = pipe(
        batch_prompts,
        do_sample=False,
        temperature=0.1,
        repetition_penalty=1.2,
        add_special_tokens=True
    )

    # 각 프롬프트에 대해 첫 번째 생성 결과를 이용하여, 원본 프롬프트 이후의 텍스트만 추출
    for prompt, outputs in zip(batch_prompts, batch_outputs):
        generated_text = outputs[0]["generated_text"][len(prompt):]
        test_results.append(generated_text)

Device set to use cuda:0


  0%|          | 0/121 [00:00<?, ?it/s]

In [96]:
# 생성된 텍스트 리스트에 대해 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

# sample_submission 파일 불러오기
submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv', encoding='utf-8-sig')

# 최종 결과 저장: 두 번째 열에 생성 텍스트, 이후 열에 임베딩 벡터 할당
submission.iloc[:, 1] = test_results
submission.iloc[:, 2:] = pred_embeddings

print(submission.head())

# 최종 결과 CSV로 저장
submission.to_csv('exaone+QLoRA_V1.csv', index=False, encoding='utf-8-sig')


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(964, 768)
         ID                                    재발방지대책 및 향후조치계획     vec_0  \
0  TEST_000  향후 작업에서는 펌프카와 아웃트리거 설치 시 지반의 안정성 확인과 함께 균일한 펼쳐... -0.913561   
1  TEST_001  모든 작업자들께 정기적인 안전교육 실시와 함께 개인 보호 장비 철저히 착용 의무화하... -0.174510   
2  TEST_002  안전 교육 강화와 함께 작업자들의 개인 보호 장비 착용 의무화하고 정기적인 위험성 ... -0.539435   
3  TEST_003  안전 교육 강화와 함께 작업 현장의 모든 표면 상태 점검 의무화하고 즉시 위험 요소... -0.322060   
4  TEST_004  안전 교육 강화와 함께 작업 중 식사 시간 지정 금지 규정 마련하고, 개인 보호 장... -0.730720   

      vec_1     vec_2     vec_3     vec_4     vec_5     vec_6     vec_7  ...  \
0 -0.739268 -0.118051  0.621568 -0.192355  1.112125  0.379940 -0.223293  ...   
1  0.282273 -0.264161  0.510851 -0.183968  0.442356  0.498790 -0.158600  ...   
2  0.232039 -0.583639  0.676231 -0.885169  0.882436  0.087071  0.261996  ...   
3  0.233395  0.088202  0.604722 -0.031129  0.576738  0.509153  0.112141  ...   
4  0.261044  0.056666  0.602097 -0.180491  0.697430 -0.157547  0.174503  ...   

    vec_758   vec_759   vec_760   vec_761   vec_762